# IBKR Execution History Retrieval
Retrieve historical trade executions from Interactive Brokers for the last 3 months

In [1]:
import time
import threading
import pandas as pd
import random
import datetime
from datetime import timedelta
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.execution import ExecutionFilter
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
class ExecutionApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.executions = []
        self.commissions = {}
        self.execution_complete = False
        
    def connectTWS(self, port=7497):
        """Connect to TWS or IB Gateway"""
        try:
            client_id = random.randint(1, 999)
            import subprocess
            try:
                result = subprocess.run(['ip', 'route', 'show'], capture_output=True, text=True)
                host_ip = None
                for line in result.stdout.split('\n'):
                    if 'default via' in line:
                        host_ip = line.split()[2]
                        break
            except:
                host_ip = '127.0.0.1'
            
            self.connect(host_ip, port, client_id)
            thread = threading.Thread(target=self.run)
            thread.daemon = True
            thread.start()
            time.sleep(2)
            
            if self.isConnected():
                logger.info(f"Connected to TWS/Gateway on {host_ip}:{port}")
                return True
            return False
        except Exception as e:
            logger.error(f"Connection error: {e}")
            return False
    
    def execDetails(self, reqId, contract, execution):
        """Callback for execution details"""
        print(f"\nExecution Received:")
        print(f"  Symbol: {contract.symbol}")
        print(f"  SecType: {contract.secType}")
        print(f"  Strike: {contract.strike if contract.secType == 'OPT' else 'N/A'}")
        print(f"  Right: {contract.right if contract.secType == 'OPT' else 'N/A'}")
        print(f"  Expiry: {contract.lastTradeDateOrContractMonth if contract.secType == 'OPT' else 'N/A'}")
        print(f"  Shares: {execution.shares}")
        print(f"  Price: ${execution.price}")
        print(f"  Time: {execution.time}")
        print(f"  Side: {execution.side}")
        print(f"  Order Ref: {execution.orderRef}")
        print(f"  Exec ID: {execution.execId}")
        
        self.executions.append({
            'execId': execution.execId,
            'orderId': execution.orderId,
            'symbol': contract.symbol,
            'secType': contract.secType,
            'strike': contract.strike if contract.secType == 'OPT' else None,
            'right': contract.right if contract.secType == 'OPT' else None,
            'expiry': contract.lastTradeDateOrContractMonth if contract.secType == 'OPT' else None,
            'shares': execution.shares,
            'price': execution.price,
            'time': execution.time,
            'side': execution.side,
            'cumQty': execution.cumQty,
            'avgPrice': execution.avgPrice,
            'orderRef': execution.orderRef,
            'permId': execution.permId,
            'acctNumber': execution.acctNumber,
            'exchange': execution.exchange,
            'evRule': execution.evRule,
            'evMultiplier': execution.evMultiplier
        })
    
    def execDetailsEnd(self, reqId):
        """Called when all executions have been received"""
        self.execution_complete = True
        print(f"\n{'='*80}")
        print(f"Execution retrieval complete: {len(self.executions)} executions received")
        print(f"{'='*80}\n")
        
    def commissionReport(self, commissionReport):
        """Callback for commission details"""
        print(f"  Commission: ${commissionReport.commission} (Exec ID: {commissionReport.execId})")
        self.commissions[commissionReport.execId] = {
            'commission': commissionReport.commission,
            'currency': commissionReport.currency,
            'realizedPNL': commissionReport.realizedPNL,
            'yield': commissionReport.yield_,
            'yieldRedemptionDate': commissionReport.yieldRedemptionDate
        }
    
    def error(self, reqId, errorCode, errorString):
        """Callback for error messages"""
        if errorCode not in [2104, 2106, 2158]:
            print(f"ERROR: reqId={reqId}, code={errorCode}, msg={errorString}")
    
    def get_executions(self, account_id, days_back=90):
        """Request executions for the specified number of days"""
        print(f"\n{'='*80}")
        print(f"Requesting executions for account {account_id}")
        print(f"Period: Last {days_back} days")
        print(f"{'='*80}\n")
        
        # Create execution filter
        exec_filter = ExecutionFilter()
        exec_filter.acctCode = account_id
        
        # Set time filter (format: yyyyMMdd-HH:mm:ss)
        start_date = datetime.datetime.now() - timedelta(days=days_back)
        exec_filter.time = start_date.strftime("%Y%m%d-00:00:00")
        
        print(f"Fetching executions from: {start_date.strftime('%Y-%m-%d')}\n")
        
        self.reqExecutions(1, exec_filter)
        
        # Wait for data
        timeout = 30
        start_time = time.time()
        while not self.execution_complete and (time.time() - start_time) < timeout:
            time.sleep(0.1)
        
        if not self.execution_complete:
            print("Warning: Timeout waiting for executions")
        
        return len(self.executions)
    
    def get_executions_dataframe(self):
        """Convert executions to DataFrame"""
        if not self.executions:
            return pd.DataFrame()
        
        df = pd.DataFrame(self.executions)
        
        # Add commission data
        df['commission'] = df['execId'].map(lambda x: self.commissions.get(x, {}).get('commission', None))
        df['realizedPNL'] = df['execId'].map(lambda x: self.commissions.get(x, {}).get('realizedPNL', None))
        
        # Convert time to datetime
        df['datetime'] = pd.to_datetime(df['time'])
        
        # Sort by time
        df = df.sort_values('datetime', ascending=False)
        
        return df
    
    def disconnect_tws(self):
        """Disconnect from TWS"""
        if self.isConnected():
            self.disconnect()

## Retrieve Executions from IBKR

In [3]:
# Initialize and connect
app = ExecutionApp()
connected = app.connectTWS()

if connected:
    # Get executions for the last 90 days (3 months)
    account_id = "DU9233079"  # Your paper trading account
    count = app.get_executions(account_id, days_back=90)
    
    print(f"\nRetrieved {count} executions")
    
    # Wait a bit for commission reports
    time.sleep(2)
    
    app.disconnect_tws()
else:
    print("Failed to connect to TWS/Gateway")

INFO:ibapi.client:sent startApi
INFO:ibapi.client:REQUEST startApi {}
INFO:ibapi.client:SENDING startApi b'\x00\x00\x00\n71\x002\x00124\x00\x00'
INFO:ibapi.wrapper:ANSWER connectAck {}
INFO:ibapi.wrapper:ANSWER nextValidId {'orderId': 1}
INFO:ibapi.wrapper:ANSWER managedAccounts {'accountsList': 'DU9233079'}
INFO:__main__:Connected to TWS/Gateway on 172.21.240.1:7497
INFO:ibapi.client:REQUEST reqExecutions {'reqId': 1, 'execFilter': 139743247372208: Object}
INFO:ibapi.client:SENDING reqExecutions b'\x00\x00\x00(7\x003\x001\x000\x00DU9233079\x0020250727-00:00:00\x00\x00\x00\x00\x00'



Requesting executions for account DU9233079
Period: Last 90 days

Fetching executions from: 2025-07-27


Execution Received:
  Symbol: META
  SecType: OPT
  Strike: 645.0
  Right: P
  Expiry: 20251024
  Shares: 1.0
  Price: $0.0
  Time: 20251025  02:51:49
  Side: SLD
  Order Ref: 
  Exec ID: 000261b8.68fb0a3d.01.01

Execution Received:
  Symbol: UNH
  SecType: OPT
  Strike: 390.0
  Right: C
  Expiry: 20251024
  Shares: 1.0
  Price: $0.0
  Time: 20251025  02:51:49
  Side: BOT
  Order Ref: 
  Exec ID: 000261b8.68fb0a3e.01.01

Execution Received:
  Symbol: UNH
  SecType: OPT
  Strike: 400.0
  Right: C
  Expiry: 20251024
  Shares: 1.0
  Price: $0.0
  Time: 20251025  02:51:49
  Side: SLD
  Order Ref: 
  Exec ID: 000261b8.68fb0a3f.01.01

Execution Received:
  Symbol: META
  SecType: OPT
  Strike: 655.0
  Right: P
  Expiry: 20251024
  Shares: 1.0
  Price: $0.0
  Time: 20251025  02:51:49
  Side: BOT
  Order Ref: 
  Exec ID: 000261b8.68fb0a40.01.01

Execution Received:
  Symbol: META
  SecType

INFO:ibapi.client:disconnecting
INFO:ibapi.wrapper:ANSWER connectionClosed {}


## View Executions as DataFrame

In [4]:
# Get DataFrame
executions_df = app.get_executions_dataframe()

if not executions_df.empty:
    print(f"Total executions: {len(executions_df)}")
    print(f"Date range: {executions_df['datetime'].min()} to {executions_df['datetime'].max()}")
    print(f"\nExecutions by security type:")
    print(executions_df['secType'].value_counts())
    print(f"\nExecutions by side:")
    print(executions_df['side'].value_counts())
    
    # Display the dataframe
    display(executions_df)
else:
    print("No executions found")

Total executions: 8
Date range: 2025-10-25 02:51:49 to 2025-10-25 02:51:49

Executions by security type:
secType
OPT    8
Name: count, dtype: int64

Executions by side:
side
SLD    4
BOT    4
Name: count, dtype: int64


,execId,orderId,symbol,secType,strike,right,expiry,shares,price,time,...,avgPrice,orderRef,permId,acctNumber,exchange,evRule,evMultiplier,commission,realizedPNL,datetime
0,000261b8.68fb0a3d.01.01,0,META,OPT,645.0,P,20251024,1.0,0.0,20251025 02:51:49,...,0.0,,1662098544,DU9233079,SMART,,0.0,None,None,2025-10-25 02:51:49
1,000261b8.68fb0a3e.01.01,0,UNH,OPT,390.0,C,20251024,1.0,0.0,20251025 02:51:49,...,0.0,,1662098545,DU9233079,SMART,,0.0,None,None,2025-10-25 02:51:49
2,000261b8.68fb0a3f.01.01,0,UNH,OPT,400.0,C,20251024,1.0,0.0,20251025 02:51:49,...,0.0,,1662098546,DU9233079,SMART,,0.0,None,None,2025-10-25 02:51:49
3,000261b8.68fb0a40.01.01,0,META,OPT,655.0,P,20251024,1.0,0.0,20251025 02:51:49,...,0.0,,1662098547,DU9233079,SMART,,0.0,None,None,2025-10-25 02:51:49
4,000261b8.68fb0a41.01.01,0,META,OPT,640.0,P,20251024,1.0,0.0,20251025 02:51:49,...,0.0,,1662098548,DU9233079,SMART,,0.0,None,None,2025-10-25 02:51:49
5,000261b8.68fb0a42.01.01,0,NKE,OPT,68.0,P,20251024,1.0,0.0,20251025 02:51:49,...,0.0,,1662098549,DU9233079,SMART,,0.0,None,None,2025-10-25 02:51:49
6,000261b8.68fb0a43.01.01,0,NKE,OPT,60.0,P,20251024,1.0,0.0,20251025 02:51:49,...,0.0,,1662098550,DU9233079,SMART,,0.0,None,None,2025-10-25 02:51:49
7,000261b8.68fb0a44.01.01,0,META,OPT,630.0,P,20251024,1.0,0.0,20251025 02:51:49,...,0.0,,1662098551,DU9233079,SMART,,0.0,None,None,2025-10-25 02:51:49


## Filter Options Only

In [5]:
# Filter for options only
if not executions_df.empty:
    options_df = executions_df[executions_df['secType'] == 'OPT'].copy()
    
    print(f"Options executions: {len(options_df)}")
    print(f"\nOptions by symbol:")
    print(options_df['symbol'].value_counts())
    
    # Create readable description
    options_df['description'] = (options_df['symbol'] + ' ' + 
                                  options_df['expiry'] + ' ' + 
                                  options_df['strike'].astype(str) + ' ' + 
                                  options_df['right'])
    
    display(options_df[['datetime', 'description', 'side', 'shares', 'price', 'commission', 'realizedPNL']])
else:
    print("No options executions found")

Options executions: 8

Options by symbol:
symbol
META    4
UNH     2
NKE     2
Name: count, dtype: int64


,datetime,description,side,shares,price,commission,realizedPNL
0,2025-10-25 02:51:49,META 20251024 645.0 P,SLD,1.0,0.0,None,None
1,2025-10-25 02:51:49,UNH 20251024 390.0 C,BOT,1.0,0.0,None,None
2,2025-10-25 02:51:49,UNH 20251024 400.0 C,SLD,1.0,0.0,None,None
3,2025-10-25 02:51:49,META 20251024 655.0 P,BOT,1.0,0.0,None,None
4,2025-10-25 02:51:49,META 20251024 640.0 P,BOT,1.0,0.0,None,None
5,2025-10-25 02:51:49,NKE 20251024 68.0 P,BOT,1.0,0.0,None,None
6,2025-10-25 02:51:49,NKE 20251024 60.0 P,SLD,1.0,0.0,None,None
7,2025-10-25 02:51:49,META 20251024 630.0 P,SLD,1.0,0.0,None,None


## Group by Order to Find Spreads

In [6]:
# Group by orderId to find spreads (multiple legs in same order)
if not executions_df.empty:
    options_df = executions_df[executions_df['secType'] == 'OPT'].copy()
    
    # Group by orderId and time (same order, same time = likely a spread)
    order_groups = options_df.groupby('orderId')
    
    spreads_found = []
    
    for order_id, group in order_groups:
        if len(group) >= 2:  # Spread has at least 2 legs
            # Check if same symbol, expiry, right (call or put)
            if (group['symbol'].nunique() == 1 and 
                group['expiry'].nunique() == 1 and 
                group['right'].nunique() == 1):
                
                symbol = group.iloc[0]['symbol']
                expiry = group.iloc[0]['expiry']
                right = group.iloc[0]['right']
                
                strikes = sorted(group['strike'].unique())
                sides = group.groupby('strike')['side'].first()
                
                spreads_found.append({
                    'orderId': order_id,
                    'symbol': symbol,
                    'expiry': expiry,
                    'right': right,
                    'strikes': f"{strikes[0]}/{strikes[1]}",
                    'datetime': group.iloc[0]['datetime'],
                    'leg_count': len(group)
                })
    
    if spreads_found:
        spreads_df = pd.DataFrame(spreads_found)
        print(f"\nFound {len(spreads_df)} potential spreads:")
        display(spreads_df)
    else:
        print("No spreads found (multiple legs in same order)")
else:
    print("No data to analyze")

No spreads found (multiple legs in same order)


## Export to CSV (Optional)

In [7]:
# Export all executions to CSV
if not executions_df.empty:
    output_file = '/home/cdodd/optcom/ibkr_executions_history.csv'
    executions_df.to_csv(output_file, index=False)
    print(f"Exported {len(executions_df)} executions to {output_file}")
    
    # Export options only
    options_output = '/home/cdodd/optcom/ibkr_options_executions.csv'
    options_df = executions_df[executions_df['secType'] == 'OPT']
    options_df.to_csv(options_output, index=False)
    print(f"Exported {len(options_df)} options executions to {options_output}")
else:
    print("No data to export")

Exported 8 executions to /home/cdodd/optcom/ibkr_executions_history.csv
Exported 8 options executions to /home/cdodd/optcom/ibkr_options_executions.csv
